In [540]:
import os
import sys


SPARK_HOME = "/usr/hdp/current/spark2-client"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["PYTHONHASHSEED"] = "42"
PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.7-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

In [541]:
from pyspark import SparkContext, SparkConf
import sys


conf = SparkConf()
sc = SparkContext(appName="hw_3_Uhbifg_1", conf=conf)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=hw_3_Uhbifg_1, master=yarn) created by __init__ at <ipython-input-515-bbbc8b4fe7cc>:6 

In [697]:
start_node = 15 # sys.argv[1]
end_node = 246 # sys.argv[2]
path_data = "/datasets/twitter/twitter.tsv"# sys.argv[3]
path_ans = "output_1"# sys.argv[4]

In [630]:
raw_graph = sc.textFile(path_data)
graph = raw_graph.map(lambda x: list(map(int, x.split("\t")[::-1]))).cache()
links = graph.groupByKey().partitionBy(sc.defaultParallelism).mapValues(list).cache()     


In [677]:
3631158 - len(a)

70

In [545]:
w = dict(links.collect())

In [714]:
from numpy.random import randint
for i in range(100):

    start_node = 21 # sys.argv[1]
    end_node = 805 # sys.argv[2]

    def search(x1, x2, x3, seen):
        ans = list()
        a = x3[x1]
        for i in a:
            if i not in seen:
                ans.append(tuple([i, x2 + [i]]))
        return ans


    links = graph.groupByKey().mapValues(list).cache()     
    answers = []
    pathes = sc.parallelize([(start_node, [start_node])])
    seen = set([start_node])
    num = 0
    while True:
        if pathes.count() == 0:
            ans = sc.parallelize([])
            break
        ans = pathes.filter(lambda x : x[0] == end_node)
        if ans.count() > 0:
            break
        else:
            rdd2 = pathes.map(lambda x : x[0], preservesPartitioning=True).distinct().collect()
            neigbours = dict() 
            for i in rdd2:
                neigbours[i] = w.get(i, [])
            pathes = sc.parallelize(pathes.flatMap(lambda x : search(x[0], x[1], neigbours, seen), preservesPartitioning=True).collect())
            seen.update(pathes.map(lambda x : x[0]).distinct().collect())
    ans = ans.map(lambda x : x[1])
    ans1 = ans.collect()
    print(ans1)

    def search(x1, x2, x3):
        ans = list()
        a = x3[x1]
        for i in a:
            ans.append(tuple([i, x2 + [i]]))
        return ans


    links = graph.groupByKey().mapValues(list).cache()     
    answers = []
    pathes = sc.parallelize([(start_node, [start_node])])
    num = 0
    while True:
        print(num)
        if num  >= 50 or pathes.count() == 0:
            ans = sc.parallelize([])
            break
        num += 1
        #print(pathes.collect())
        ans = pathes.filter(lambda x : x[0] == end_node)
        if ans.count() > 0:
            break
        else:
            rdd2 = pathes.map(lambda x : x[0], preservesPartitioning=True).distinct().collect()
            neigbours = dict() 
            for i in rdd2:
                neigbours[i] = w.get(i, [])
            pathes = sc.parallelize(pathes.flatMap(lambda x : search(x[0], x[1], neigbours), preservesPartitioning=True).collect())
    ans = ans.map(lambda x : x[1])

    ans2 = ans.collect()

    print(ans1 == ans2)

[]
0
1
2
3
4


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 8547.0 failed 4 times, most recent failure: Lost task 1.3 in stage 8547.0 (TID 28735, name2.ru-central1.internal, executor 27): ExecutorLostFailure (executor 27 exited caused by one of the running tasks) Reason: Container from a bad node: container_e16_1615841176380_0979_01_000030 on host: name2.ru-central1.internal. Exit status: 143. Diagnostics: [2021-03-30 03:45:15.806]Container killed on request. Exit code is 143
[2021-03-30 03:45:15.807]Container exited with a non-zero exit code 143. 
[2021-03-30 03:45:15.809]Killed by external signal
.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1925)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1913)
	at org.a

In [711]:
end_node 

805

In [698]:

def search(x1, x2, x3, seen):
    ans = list()
    a = x3[x1]
    for i in a:
        if i not in seen:
            ans.append(tuple([i, x2 + [i]]))
    return ans


links = graph.groupByKey().mapValues(list).cache()     
answers = []
pathes = sc.parallelize([(start_node, [start_node])])
seen = set([start_node])
num = 0
while True:
    if pathes.count() == 0:
        ans = sc.parallelize([])
        break
    ans = pathes.filter(lambda x : x[0] == end_node)
    if ans.count() > 0:
        break
    else:
        rdd2 = pathes.map(lambda x : x[0], preservesPartitioning=True).distinct().collect()
        neigbours = dict() 
        for i in rdd2:
            neigbours[i] = w.get(i, [])
        pathes = sc.parallelize(pathes.flatMap(lambda x : search(x[0], x[1], neigbours, seen), preservesPartitioning=True).collect())
        seen.update(pathes.map(lambda x : x[0]).distinct().collect())
ans = ans.map(lambda x : x[1])

In [723]:
def search(x1, x2, x3, seen):
    ans = list()
    for i in x3:
        if i not in seen:
            ans.append(tuple([i, x2 + [i]]))
    return ans

num = 0
links = graph.groupByKey().mapValues(list).cache()     
answers = []
pathes = sc.parallelize([(start_node, [start_node])])
seen = set([start_node])
while True:
    num += 1
    print(num)
    if pathes.count() == 0:
        ans = sc.parallelize([])
        break
    ans = pathes.filter(lambda x : x[0] == end_node)
    if ans.count() > 0:
        break
    else:
        pathes = sc.parallelize(pathes.flatMap(lambda x : search(x[0], x[1], w.get(x[0], []), seen), preservesPartitioning=True).collect())
        seen.update(pathes.map(lambda x : x[0]).distinct().collect())
ans = ans.map(lambda x : x[1])

1
2
3
4
5
6
7
8


In [720]:
def search(x1, x2, x3, seen):
    ans = list()
    a = x3[x1]
    for i in a:
        if i not in seen:
            ans.append(tuple([i, x2 + [i]]))
    return ans

num = 0
links = graph.groupByKey().mapValues(list).cache()     
answers = []
pathes = sc.parallelize([(start_node, [start_node])])
seen = set([start_node])
while True:
    num += 1
    print(num)
    if pathes.count() == 0:
        ans = sc.parallelize([])
        break
    ans = pathes.filter(lambda x : x[0] == end_node)
    if ans.count() > 0:
        break
    else:
        rdd2 = pathes.map(lambda x : x[0], preservesPartitioning=True).distinct().collect()
        neigbours = dict() 
        for i in rdd2:
            neigbours[i] = w.get(i, [])
        pathes = sc.parallelize(pathes.flatMap(lambda x : search(x[0], x[1], neigbours, seen), preservesPartitioning=True).collect())
        seen.update(pathes.map(lambda x : x[0]).distinct().collect())
ans = ans.map(lambda x : x[1])

1
2
3
4
5
6
7
8


In [724]:
ans.collect()

[]

In [595]:
path_ans = "1.csv"

In [599]:
def toCSVLine(data):
    return ','.join(str(d) for d in data)

lines = ans.map(toCSVLine)
lines.coalesce(1).saveAsTextFile(path_ans)

In [602]:
read_rdd = sc.textFile("Uhbifg_hw3_output")


In [603]:
read_rdd.collect()

[]

In [553]:
lines.collect()

['12,14,34', '12,21,34', '12,23,34', '12,295,34', '12,322,34']

In [557]:
!hdfs dfs -rm -rf 1.csv

-rm: Illegal option -rf
Usage: hadoop fs [generic options]
	[-appendToFile <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-copyFromLocal [-f] [-p] [-l] [-d] [-t <thread count>] <localsrc> ... <dst>]
	[-copyToLocal [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-count [-q] [-h] [-v] [-t [<storage type>]] [-u] [-x] [-e] <path> ...]
	[-cp [-f] [-p | -p[topax]] [-d] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] [-v] [-x] <path> ...]
	[-expunge]
	[-find <path> ... <expression> ...]
	[-get [-f] [-p] [-ignoreCrc] [-crc] <src> ... <localdst>]
	[-getfacl [-R] <path>]
	[-getfattr [-R] {-n name | -d} [-e en] <path>]
	[-getmerge [-nl] [-skip-empty-file] <src> <localdst>]
	[-head <file>]
	[-help [cmd ...]]
	[-ls [-C] [-d] [-h] [-q] [

In [601]:
!hdfs dfs -ls  Uhbifg_hw3_output

Found 3 items
-rw-r--r--   3 Uhbifg Uhbifg          0 2021-03-30 02:24 Uhbifg_hw3_output/_SUCCESS
-rw-r--r--   3 Uhbifg Uhbifg          0 2021-03-30 02:24 Uhbifg_hw3_output/part-00000
-rw-r--r--   3 Uhbifg Uhbifg          0 2021-03-30 02:24 Uhbifg_hw3_output/part-00001


In [587]:
!hdfs dfs -cat 1.csv

cat: `1.csv': Is a directory


In [600]:
!hdfs dfs -cat 1.csv/part-00000

In [598]:
!hdfs dfs -rm -r -f 1.csv

21/03/30 03:01:03 INFO fs.TrashPolicyDefault: Moved: 'hdfs://name1.ru-central1.internal:8020/user/Uhbifg/1.csv' to trash at: hdfs://name1.ru-central1.internal:8020/user/Uhbifg/.Trash/Current/user/Uhbifg/1.csv1617073263496


In [597]:
!hdfs dfs -ls 1.csv

Found 2 items
-rw-r--r--   3 Uhbifg Uhbifg          0 2021-03-30 02:56 1.csv/_SUCCESS
-rw-r--r--   3 Uhbifg Uhbifg         47 2021-03-30 02:56 1.csv/part-00000
